In [0]:
import torch
import gym

import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# import torch.distributions.categorical
from torch.distributions.categorical import Categorical

import matplotlib.pyplot as plt

In [2]:
env = gym.make('CartPole-v1')

state_size = env.observation_space.shape[0]
action_size  =env.action_space.n


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
GAMMA=0.99

class PolicyNetwork(nn.Module):
    def __init__(self,state_size,action_size):
        super(PolicyNetwork, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(state_size,128),
            nn.Dropout(p=0.6),
            nn.ReLU(),
            nn.Linear(128,action_size),
            nn.Softmax(), 


        )
    def forward(self,x):
        x = torch.FloatTensor(x)
        output = self.layers(x)
        
        return output

In [0]:
class ReinforceAgent:
    def __init__(self,state_size,action_size):
        self.policy_net = PolicyNetwork(state_size,action_size)
        self.optimizer = optim.Adam(self.policy_net.parameters(),lr=1e-2)

        self.state_size = state_size
        self.action_size = action_size
        

    #this should be ok
    def get_action(self,state):
        state = torch.FloatTensor(state)
        #this will return probabilities from which we will sample
        probs = self.policy_net.forward(state) 
        # print(probs)

        #one way of sampling using numpy
        # probs_np = log_probs.detach().numpy()
        # action = np.random.choice(range(action_size),p = probs_np)

        #second way of sampling using pytorch(logits will be converted into probs)
        sampler = Categorical(probs=probs)
        action = sampler.sample()
        
        log_prob = sampler.log_prob(action)
        # print(log_prob)

        return action.item(),log_prob

    #here i update policy_network 
    def update_policy(self,log_probs,rewards): #states and actions should be torch.FloatTensor
        #compute logits(outputs from NN)
        # logits = self.policy_net.forward(states)
        #make a Categorical to be able to sample
        # sampler = Categorical(logits)
        #compute logP(A|state)
        # logp = sampler.log_prob(actions)

        #compute mean loss for the whole episode

        rewards = discount(rewards)
        loss = []
        for logp,reward in zip(log_probs,rewards):
            loss.append(-logp*reward)
        
        loss = torch.stack(loss)

        # we should use mean loss, but it doesn't work and i don't know why
        # loss = loss.mean()
        loss = loss.sum()
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

def discount(rewards,GAMMA=1):
    #this is like dynamic programming
    R = 0
    discounted_rewards=rewards.copy()
    #discounting rewards 
    for r in rewards[::-1]:
        R = r + GAMMA*R
        #it appends to the beginning of the list, first value is the index of element before which we want to append
        discounted_rewards.insert(0,R)

    discounted_rewards = torch.tensor(discounted_rewards)

    #normalizing (x-mean(x))/std
    discounted_rewards = (discounted_rewards - discounted_rewards.mean())/(discounted_rewards.std() + 1e-9)

    return discounted_rewards


In [0]:
agent = ReinforceAgent(state_size,action_size)

In [6]:
max_episode_num = 15000
max_steps = 10000

def training():
    avg_rewards = []
    #main loop aka epochs
    for episode in range(1):
        state = env.reset()

        rewards=[]
        weights=[]
        actions=[]

        
        log_probs=[]
        while True:

            action,log_prob = agent.get_action(state)
            
            actions.append(action)
            log_probs.append(log_prob)
            
            new_state,reward, done, _ = env.step(action)
            
            state = new_state
            rewards.append(reward)
            
            if done:

                log_probs = torch.stack(log_probs)


                agent.update_policy(log_probs,rewards)
                avg_rewards.append(sum(rewards))

                if episode % 500 == 0:
                    avg_rew = sum(avg_rewards)/len(avg_rewards)
                    print("episode: {}, episode reward: {}, average reward: {}".format(episode, sum(rewards), avg_rew))

                
                break

            
training()


episode: 0, episode reward: 22.0, average reward: 22.0


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
